# Skforecast in GPU

Traditionally, machine learning algorithms are executed on CPUs (Central Processing Units), which are general-purpose processors that are designed to handle a wide range of tasks. However, CPUs are not optimized for the highly parallelized matrix operations that are required by many machine learning algorithms, which can result in slow training times and limited scalability. GPUs, on the other hand, are designed specifically for parallel processing and can perform thousands of mathematical operations simultaneously, making them ideal for training and deploying large-scale machine learning models.

Three popular machine learning libraries that have implemented GPU acceleration are **XGBoost**, **LightGBM** and **CatBoost**. These libraries are used for building gradient boosting models, which are a type of machine learning algorithm that is highly effective for a wide range of tasks, including forecasting. With GPU acceleration, these libraries can significantly reduce the training time required to build these models and improve their scalability.

Despite the significant advantages offered by GPUs (specifically Nvidia GPUs) in accelerating machine learning computations, access to them is often limited due to high costs or other practical constraints. Fortunatelly, **Google Colaboratory (Colab)**, a free Jupyter notebook environment, allows users to run Python code in the cloud, with access to powerful hardware resources such as GPUs. This makes it an excellent platform for experimenting with machine learning models, especially those that require intensive computations.

The following sections demonstrate how to install and use **XGBoost** and **LightGBM** with GPU acceleration to create powerful forecasting models.

<div class="admonition note" name="html-admonition" style="background: rgba(0,184,212,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00b8d4; border-color: #00b8d4; padding-left: 10px; padding-right: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00b8d4;"></i>
    <b style="color: #00b8d4;">&#9998 Note</b>
</p>

<p>The following code assumes that the user is executing it in Google Colab with an activated GPU runtime.</p>
<ul>
<li><a href="https://colab.research.google.com/drive/10PYQFQN9oNkAHh0X7wwyBLQ3JQ_Cm7pP?usp=sharing">Skforecast in GPU: XGBoost</a></li>
<li><a href="https://colab.research.google.com/drive/17Csc70AY-GQA-tvZjq9TYCbmnrNOzslh?usp=sharing">Skforecast in GPU: LightGBM</a></li>
<li><a href="https://colab.research.google.com/drive/1Z-n0kKEnQvY02e9-HxKbkTdLc10RNd_-?usp=sharing">Skforecast in GPU: LightGBM</a></li>
</ul>

</div>

## XGBoost

When creating the model with XGBoost version >= 2.0, two arguments are need to indicate XGBoost to run in GPU, if it available: `device='cuda'` and `tree_method='hist'`.

In [9]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent.parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\jaesc2\GitHub\skforecast


In [10]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import time
from xgboost import XGBRegressor
import torch
import os
import sys
import psutil
from skforecast.recursive import ForecasterRecursive

In [11]:
# Print information abput the GPU and CPU
# ==============================================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated :', round(torch.cuda.memory_allocated(0) / 1024**3, 1), 'GB')
    print('Reserved  :', round(torch.cuda.memory_reserved(0) / 1024**3, 1), 'GB')

print(f"CPU RAM Free: {psutil.virtual_memory().available / 1024**3:.2f} GB")

Using device: cuda
NVIDIA T1200 Laptop GPU
Memory Usage:
Allocated : 0.0 GB
Reserved  : 0.0 GB
CPU RAM Free: 10.97 GB


In [6]:
# Data
# ==============================================================================
data = pd.Series(np.random.normal(size=1000000))

In [7]:
# Create and train forecaster with a XGBRegressor using GPU
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor = XGBRegressor(
                                 n_estimators = 5000,
                                 tree_method  = 'hist',
                                 device       = 'cuda'
                             ),
                 lags = 20
             )

forecaster.fit(y=data)

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:25:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [8]:
# Create and train forecaster with a XGBRegressor using GPU
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor = XGBRegressor(
                                 n_estimators = 5000,
                                 tree_method  = 'hist',
                             ),
                 lags = 20
             )

forecaster.fit(y=data)

## LightGBM

In [14]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import time
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursive

In [12]:
# Data
# ==============================================================================
data = pd.Series(np.random.normal(size=1000000))

In [17]:
# Create and train forecaster with a LGBMRegressor using GPU
# ==============================================================================
regressor = LGBMRegressor(
    boosting_type='gbdt',
    device='gpu',
    n_estimators=5000,
    learning_rate=0.1
)

forecaster = ForecasterRecursive(
                 regressor = regressor,
                 lags      = 20
             )

forecaster.fit(y=data)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5100
[LightGBM] [Info] Number of data points in the train set: 999980, number of used features: 20
[LightGBM] [Info] Using GPU Device: NVIDIA T1200 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 20 dense feature groups (19.07 MB) transferred to GPU in 0.027916 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score -0.000370


KeyboardInterrupt: 